# Generator

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
data_path = '/content/drive/Shareddrives/DeepLearning_2022/Final-Project/Data/'
model_path = '/content/drive/Shareddrives/DeepLearning_2022/Final-Project/models/'

Mounted at /content/drive/


In [2]:
import torch
import torch.nn as nn
import torch
def vocabulary(words):
  """
  Given a plain text in a list returns a list with all the different words.
  """
  vocab = set()
  for word in words:
    if word not in vocab:
      vocab.add(word)

  return list(vocab)

#Decide what to do with words that are not in the vocabulary
def text_to_seq(text,vocab,add_n = False):
  """
  Given a text split it and a vocabulary, return a list of numbers that refers 
  to the index position of each word in the vocabulary.
  """
  sequence = []
  for word in text:
    if word != "":
      sequence.append(vocab.index(word))

  return torch.tensor(sequence)

with open(data_path + "llibres_dataset_v3.txt","r") as f:
  dataset = ""
  for line in f:
    dataset += line 

for_vocabulary = dataset.replace("\n"," ")
vocab = vocabulary(for_vocabulary.split(" "))
print("Number of different words",len(vocab))

Number of different words 30592


In [28]:
class LSTM_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,dropout):
        super(LSTM_model, self).__init__()
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(input_size, hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        #self.sigm = nn.Sigmoid()
        #self.dropout = nn.Dropout(dropout)

    def zero_hidden(self):
        # Initialize lstm states with zeros
        return (torch.zeros(1, 1, hidden_dim).cuda(),
                torch.zeros(1, 1, hidden_dim).cuda())

    def forward(self, input):
        x = self.embed(input).unsqueeze(1) #We need the unsqueze to introduce the new dimension
        output_lstm, states_lstm = self.lstm1(x, self.zero_hidden())
        output_lstm, states_lstm = self.lstm2(output_lstm, self.zero_hidden())
        #output_lstm = self.dropout(output_lstm)
        output_fc = self.fc(output_lstm)
        #out = self.sigm(output_fc)
        return output_fc, output_lstm, states_lstm

In [29]:
embedding_dim = 400
hidden_dim = 400
vocab_size = len(vocab)
#vocab_size = 14379
LSTM_model = LSTM_model(embedding_dim, hidden_dim, vocab_size,dropout=None).cuda()

In [30]:
LSTM_model.load_state_dict(torch.load(model_path+'/language_generator.ckpt'))

<All keys matched successfully>

In [31]:
def tensor_to_str(tensor,vocab):
  text = ""
  for elem in tensor:
    text += " "+ vocab[int(elem)]
  return text


In [73]:

sig = nn.Softmax()

def generator(model,n_words,vocab,seed):
  result = text_to_seq(seed.split(" "),vocab).cuda()
  final_list = list(result)
  for n in range(n_words):
    #Instead of max sample
    #print(torch.tensor(final_list))
    val = model(torch.tensor(final_list).cuda())[0][0]
    val = sig(val)
    final_list.append(val.multinomial(num_samples=1))
  return tensor_to_str(final_list,vocab)
seed = "estic"
print(generator(LSTM_model,5,vocab,seed))

 estic menu grinyolant germanivolament peculiar pets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Input / Output
#a mi m agraden els llibres perque soc / automata

# el ocell / oest despatxava absorcions ficantse

#tu ets / desplaent

# tu ets / becker pressentia

# nosaltres som / clausules meno ario molestaven emmidonats